# users table에 패스워드 넣기

In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

import pymysql
conn = pymysql.connect(
    host='132.226.20.89',
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
)

## 패스워드 암호화

- Secure Hash Algorithm(SHA-256) base64
- 단방향과 양방향이 있음. 양방향은 평문 비문 왔다갔다 가능. 단방향은 비문에서 평문으론 irreversible

In [4]:
# 단방향으로 평문을 비문으로 바꿈
import hashlib

pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩

In [5]:
import base64

base64.b64encode(pwd_sha256.digest())
# bite

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [6]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd
# string    데이터 타입이 다름

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [7]:
len(hashed_pwd)

44

- 패스워드를 생성하는 함수

In [9]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [10]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기
- uid + 21

In [16]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [17]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))
conn.commit()

- email 추가: uid@mc.com

In [19]:
sql_email = 'update users set email=%s where uid=%s;'
for line in results:
    uid = line[0]
    email = uid+ '@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

## 로그인

In [ ]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

- Case 1) uid가 없는 경우

In [20]:
# result가 Null
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result      # null값이 나옴

- Case 2) 비밀번호가 틀린 경우

In [22]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result      # uid는 결과가 나왔으니 맞음

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [23]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Incorrect password!


- Case 3) OK

In [26]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Login


- 3가지 경우를 합침

In [27]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Login


In [28]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Incorrect password!


In [29]:
uid, pwd = 'djy1', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Bad uid!!!


- 예외 처리를 통해 해결하는 경우

In [30]:
uid, pwd = 'djy1', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Bad uid!!!


In [32]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Incorrect password!


In [31]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Login


## 마무리

In [33]:
cur.close()
conn.close()